# Despliegue de modelos para inferencia online

En esta primera práctica vamos a aprender cómo desplegar un modelo para realizar inferencias online usando un microservicio. Para ello, utilizaremos Google Cloud Platform (GCP).

El objetivo de esta práctica es dado un modelo ya entrenado, construir un microservicio capaz de disponibilizar nuestro modelo a gran escala para peticiones en tiempo real.

El modelo lo tendremos almacenado en Google Cloud Storage y generaremos nuestro microservicio usando FastAPI y lo desplegaremos un el servicio Serverless de GCP, Cloud Run.

Al terminar esta práctica seremos capaces de crear microservicios de Machine Learning para poner en inferencia online los modelos que deseemos a gran escala.


# Para empezar... ¿Qué es una API?

Para construir aplicaciones que sean escalables e interactivas, es necesario que éstas sean capaces de comunicarse entre ellas. Por tanto, una API (abreviatura de Application Programming Interface) son una serie de reglas que facilitan las comunicaciones entre aplicaciones. Estas aplicaciones pueden ser librerías de Python o servidores web entre otros.

Una de las principales ventajas de una API es que el solicitante no necesita saber el funcionamiento interno de la aplicación ni el lenguaje en el que esté desarrollado para poder responder y viceversa. Esto permite que diferentes servicios que usen diferentes tecnologías se comuniquen de una manera estándar.

# Introducción a FastAPI

[FastAPI](https://fastapi.tiangolo.com/) es un framework web de alto rendimiento para la construcción de APIs en Python 3.6+. Es uno de los frameworks más completos para el uso de APIs en producción vía Python.

In [ ]:
! pip install fastapi[all] pyngrok streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 

In [ ]:
!pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%writefile main.py

from fastapi import FastAPI #importamos el modulo bascio

app = FastAPI()  #creamos la aplicacion (la API) instanciando el objeto


@app.get("/saluda") #añadimos una ruta que le permite comunicarse con otros servicios. Es una cláusula para obtención de información
async def root():
    return {"message": "Hello World"}

Writing main.py


Las siguientes celdas permiten ejecutar un microservicio en colab. Si deseáis reproducirlo os tenéis que generar un token en [ngrok](https://dashboard.ngrok.com/get-started/your-authtoken).

La primera línea importa FastAPI. Tras lo cual se crea una APP vacía y se añade una consulta get. Las consultas get buscan devolver información. Todo esto se escribe en un fichero. Podemos probarlo con lo siguiente:

In [ ]:
import nest_asyncio
from pyngrok import ngrok, conf

conf.get_default().auth_token = "" #cada uno os lo debéis generar en ngrok.

ngrok_tunnel = ngrok.connect(8000) #8000 es el puerto en el que le indicamos que escuche
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

Public URL: http://6395-35-203-159-21.ngrok.io


Esto genera un tunel de la URL pública a la interna de la máquina. Mediante la siguiente celda podemos observar la traza de las llamadas que se hacen y llamar a la API

In [ ]:
! uvicorn main:app --port 8000

INFO:     Started server process [1476]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     83.32.134.84:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     83.32.134.84:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     83.32.134.84:0 - "GET /saluda HTTP/1.1" 200 OK
INFO:     83.32.134.84:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     Shutting down
INFO:     Finished server process [1476]
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/starlette/routing.py", line 686, in lifespan
    await receive()
  File "/usr/local/lib/python3.9/dist-packages/uvicorn/lifespan/on.py", line 139, in receive
    return await self.receive_queue.get()
  File "/usr/lib/python3.9/asyncio/queues.py", line 166, in get
    await getter
asyncio.exceptions.CancelledError



Los 404 salen porque no hay nada definido en esas rutas.

Si pones esa URL en el ordenador debería lanzar el mensaje. (Puede surgir un problema de seguridad, aceptamos). Una vez en la URL añadimos la / y con eso ya lo tendremos.

Veamos un ejemplo algo más complejo:

In [ ]:
%%writefile mainpost.py

from typing import Optional

from fastapi import FastAPI
from pydantic import BaseModel

class Indentity(BaseModel): #los identity están compuestos de un nombre obligatorio y un apellido opcional
    name: str
    surname: Optional[str] = None

app = FastAPI()

@app.get("/saluda")
async def root():
    return {"message": "Hello World"}

@app.post("/testing") #los metodos post reciben informacion
async def testing(id: Indentity): #la API espera un input de tipo Identity. Si no cumple falla
    if id.surname is None: # si no hay apellido, devuelve solo el nombre
      message = f"Welcome to the API! My name is {id.name}"
    else: message = f"Welcome to the API! My name is {id.name} {id.surname}" # si vienen los dos devuelven los dos
    return {"message": message}

Writing mainpost.py


Solo acepta la petición si se cumplen los requisitos de entrada, si no se rechaza directamente.

In [ ]:
import nest_asyncio
from pyngrok import ngrok, conf

conf.get_default().auth_token = "" #cada uno os lo debéis generar en ngrok.

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

Public URL: http://2be5-35-203-159-21.ngrok.io


Si abrimos la URL no aparece directamente. Podemos usar [Postman](https://www.postman.com/) para lanzar peticiones a la API:

In [ ]:
! uvicorn mainpost:app --port 8000

INFO:     Started server process [3353]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     83.32.134.84:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     83.32.134.84:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     83.32.134.84:0 - "GET /saluda HTTP/1.1" 200 OK
INFO:     83.32.134.84:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     83.32.134.84:0 - "GET /testing HTTP/1.1" 405 Method Not Allowed
INFO:     54.86.50.139:0 - "GET /saluda HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /testing HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /testing HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /testing HTTP/1.1" 422 Unprocessable Entity
INFO:     Shutting down
INFO:     Finished server process [3353]
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/starlette/routing.py", line 686, in lifespan
    await receive()
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ¡Manos a la obra!

Ahora vamos a comenzar con el desarrollo de la API para inferencias online. Para cada petición que recibamos de predicción vamos a realizar tres pasos:

1.   **Preprocesamiento**: en este primer paso extraeremos el dato a inferir de la petición y aplicaremos el preprocesado necesario
2.   **Inferencia**: realizaremos la inferencia sobre nuestro modelo.
3. **Postprocesado**: generaremos un JSON de respuesta con el resultado de la inferencia

**INTRODUCIR DIAGRAMA DE LA PRÁCTICA**

## Configuración de nuestro proyecto en GCP

**Los siguientes pasos es obligatorio realizarlos para seguir con la práctica.**

1.   Selecciona o crea un proyecto en GCP
2.   Asegurate de que la facturación está activada para tu proyecto.
3.   [Habilita la API de Google Cloud Storage](https://console.cloud.google.com/apis/library/storage-component.googleapis.com?q=storage).
4. [Habilita la API de Google Cloud Registry](https://console.cloud.google.com/apis/library/containerregistry.googleapis.com?q=container).
5. [Habilita la API de Google Cloud Run](https://console.cloud.google.com/apis/library/run.googleapis.com?q=cloud%20run).
6. [Habilita la API de Google Cloud Build](https://console.cloud.google.com/apis/library/cloudbuild.googleapis.com?q=cloud%20build).
7. [Habilita la API de App Engine Flexible Environment](https://console.cloud.google.com/apis/library/appengineflex.googleapis.com?q=app%20eng).
8. [Habilita la API de App Engine Admin](https://console.cloud.google.com/apis/library/appengine.googleapis.com?q=app%20engine).
9. Introduce tu ID de proyecto de GCP en la celda de abajo. Ejecuta la celda para asegurarnos de que el Cloud SDK usa el proyecto adecuado para todos los comandos en este notebook.

**Nota**: Jupyter ejecuta las lineas con el prefijo `!` como comandos shell de consola, y puede usar variables de Python en los comandos añadiendoles el prefijo `$`.

In [ ]:
PROJECT_ID = "puestaproduccionkcmarzo23" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [ ]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''


## Creación bucket en Cloud Storage

**Los siguientes pasos son obligatorios.**

Cuando ejecutemos un job de entrenamiento usando el Cloud SDK, lo que hacemos es subir un paquete Python que contiene el código de entrenamiento a Google Cloud Storage. AI Platform ejecuta este paquete en el job.

Establece el nombre del bucket a continuación. El nombre tiene que ser único para todos los bucket de GCP. También tenemos que establecer la variable `REGION`, la cual usaremos para todas las operaciones a lo largo del notebook. Asegurate de [elegir una región en la que Cloud AI Platform esté disponible](https://cloud.google.com/ml-engine/docs/tensorflow/regions).

In [ ]:
BUCKET_NAME = "puestaproduccionkcmarzo23-art" #@param {type:"string"}
REGION = "europe-west1" #@param {type:"string"}

**Sólo si tu bucket aún no existe**: Ejecuta la siguiente celda para crear tu bucket en Cloud Storage.

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finalmente, validamos que tenemos acceso al bucket de Cloud Storage mirando sus contenidos:

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

                                 gs://puestaproduccionkcmarzo23-art/data/
                                 gs://puestaproduccionkcmarzo23-art/twitter-sentiment-batch/


Aparece la carpeta de batch ergo todo correcto

## Descarga de la plantilla de código

Ahora nos descargaremos la plantilla de código que vamos a ir rellenando para el desarrollo de la práctica y establecemos el directorio como directorio de trabajo:

In [ ]:
# Clone the repository
! git clone https://github.com/ArturoSanchezPalacio/TwitterOnline.git

# Set the working directory to the sample code directory
%cd ./TwitterOnline

Cloning into 'TwitterOnline'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 27 (delta 1), reused 27 (delta 1), pack-reused 0
Unpacking objects: 100% (27/27), 5.87 KiB | 1001.00 KiB/s, done.
/content/TwitterOnline


In [ ]:
!ls

app  app.py  Dockerfile  README.md  requirements.txt


## Instalación de dependencias

Ejecutamos la siguiente celda para instalar las dependencias de Python necesarias para entrenar el modelo localmente y preprocesar datos. 

Cuando ejecutemos el job de entrenamiento en AI Platform, las dependencias estarán instaladas en base a la [versión del runtime](https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list) elegido.

In [ ]:
! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 KB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.0 MB/s eta 0:00:00
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user


In [ ]:
# Nos aseguramos que nuestras variables de entorno no hayan desaparecido si hemos reiniciado el kernel

print(f"Project: {PROJECT_ID}")

Project: puestaproduccionkcmarzo23


# Desarrollo del microservicio de inferencia

En la plantilla de código se proporciona una estructura de proyecto genérica para cualquier desarrollo de una API de inferencia online lista para ser usada de manera productiva.

El proyecto tiene la siguiente estructura:

``` bash
twitter-sentiment-online/
├── app/
│   ├── __init__.py
│   ├── api/
│   │   ├── __init__.py
│   │   └── routes/
│   │       ├── __init__.py
│   │       ├── heartbeat.py
│   │       ├── prediction.py
│   │       └── router.py
│   ├── core/
│   │   ├── __init__.py
│   │   ├── config.py
│   │   ├── enums.py
│   │   ├── event_handlers.py
│   │   └── messages.py
│   ├── main.py
│   ├── models/
│   │   ├── __init__.py
│   │   ├── heartbeat.py
│   │   ├── payload.py
│   │   └── prediction.py
│   └── services/
│       ├── __init__.py
│       └── models.py
├── Dockerfile
├── README.md
└── requirements.txt
```

En esta estructura distinguimos los siguientes componentes:

* **README.md**: instrucciones de uso.
* **Dockerfile**: aquí definimos la imagen base que usaremos y como empaquetamos el proyecto.
* **requirements.txt**: especificación de dependencias a instalar en el microservicio. Indica lo que va a construir.
* **app**: aplicación de inferencia online en FastAPI. Toda la aplicación que estamos desarrollando se encuentra en este fichero.
    * **main.py**: punto de entrada para la ejecución de la aplicación. Incluye todas las rutas de la API así como los servicios utilizados por la misma.
    * **models**: en este encontramos la definición de los esquemas que usaremos dentro de la aplicación. No hace referencia a modelos de Machine Learning sino a modelos de datos, es decir, a esquemas de la información. Sería como la clase Identity del ejemplo previo.
    * **services**: en este módulo incluiremos la implementación de nuestra clase de inferencia. Es la parte del código más personalizada. Aquí incluiremos la carga del modelo, la generación de la predicción o el formateo de los datos de salida.
    * **api/routes**: módulo en el que definiremos los diferentes endpoints que tendrá la API, son las rutas de acceso a la información.
    * **api/core**: módulo donde estarán funcionalidades comunes al servicio y configuraciones. En él almacenamos funciones que se reutilizan.

In [ ]:
%cd /content/TwitterOnline/

/content/TwitterOnline


In [ ]:
!ls

app  app.py  Dockerfile  README.md  requirements.txt


Defino la ruta en la que se encuentra el modelo que voy a usar para realizar las inferencias:

In [ ]:
import os

os.environ["DEFAULT_MODEL_PATH"] = "/content/TwitterOnline/"

Nos bajamos los archivos que ya hemos generado previamente

In [ ]:
!ls

app  app.py  Dockerfile  README.md  requirements.txt


In [ ]:
! gsutil -m cp \
  "gs://puestaproduccionkcmarzo23-art/twitter-sentiment-batch/data/model/model.h5" \
  "gs://puestaproduccionkcmarzo23-art/twitter-sentiment-batch/data/model/tokenizer.pkl" \
  .

Copying gs://puestaproduccionkcmarzo23-art/twitter-sentiment-batch/data/model/model.h5...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://puestaproduccionkcmarzo23-art/twitter-sentiment-batch/data/model/tokenizer.pkl...
| [2/2 files][355.5 MiB/355.5 MiB] 100% Done                                    
Operation completed over 2 objects/355.5 MiB.                                    


Comprobamos que están aparecen a la izquierda:

Ahora exploremos el código con el que vamos a trabajar.

Para poder probar el correcto funcionamiento del servicio en local:

In [ ]:
!ls

app  app.py  Dockerfile  model.h5  README.md  requirements.txt	tokenizer.pkl


In [ ]:
import nest_asyncio
from pyngrok import ngrok, conf

conf.get_default().auth_token = ""

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

Public URL: http://3011-104-196-115-8.ngrok.io


In [ ]:
!pip install loguru

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 2.4 MB/s eta 0:00:00


In [ ]:
! uvicorn app.main:app --port 8000

2023-03-14 21:49:39.620156: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/usr/lib/python3.9/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "uvloop/loop.pyx", line 1511, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1504, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1377, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 555, in uvloop.loop.Loop._run
  File "uvloop/loop.pyx", line 474, in uvloop.loop.Loop._on_idle
  File "uvloop/cbhandles.pyx", line 83, in uvloop.loop.Handle._run
  File "uvloop/cbhandles.pyx", line 61, in uvloop.loop.Handle._run
  File "/usr/local/lib/python3.9/dist

## Creando la interfaz usando Streamlit

[Streamlit](https://www.streamlit.io/) es un framework para la creación de Webapps orientado a datos e Inteligencia Artificial basado en Python. Echemos un ojo.

En este caso vamos a desarrollar un pequeño frontal para poder invocar nuestro recién desarrollado servicio de inferencia para realizar predicciones.

In [ ]:
%mkdir /content/prediction-front

In [ ]:
%cd /content/prediction-front

/content/prediction-front


Veamos un primer ejemplo de un front:

In [ ]:
%%writefile front.py

import requests
import validators
import streamlit as st
import pandas as pd

st.title("Predicciones Análisis Sentimiento KeepCoding")

st.markdown("¡Bievenid@! Vamos a crear un front para poder ves las predicciones de nuestro modelo :smile:") #ponemos este titulo

st.write("Debemos introducir la URL donde se servirán las predicciones de nuestro modelo") #escribimos este mensaje

server_url = st.text_input("URL predicciones", value="") #vacio por defecto, si no se almacena la URL en la variable

if server_url !="":
  st.write(f"URL: {server_url}")

Writing front.py


In [ ]:
import nest_asyncio
from pyngrok import ngrok, conf

conf.get_default().auth_token = ""

ngrok_tunnel = ngrok.connect(8501)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

Public URL: http://59fd-104-196-115-8.ngrok.io


In [ ]:
! streamlit run front.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://104.196.115.8:8501

  Stopping...
^C


Podemos parar la URL para ver un ejemplo más completo

El objetivo del siguiente script es levantar un front capaz de recibir un texto y elaborar una predicción sobre nuestro modelo:

In [ ]:
%%writefile front.py

import requests
import validators
import streamlit as st
import pandas as pd


class Session:
    pass


@st.cache(allow_output_mutation=True)
def fetch_session():
    session = Session()
    session.url = None
    session.predictions = None
    return session


session_state = fetch_session()


def validate_url(server_url): #comprueba que el texto introducido es una URL
    info_placeholder = st.empty()

    if server_url == "":
        # wait
        info_placeholder.write("")
    elif validators.url(server_url):
        # Save url
        session_state.url = server_url
        info_placeholder.write(
            "Perfect! Now we can start predicting. Insert your desired text below to make predictions:"
        )
    else:
        info_placeholder.write("Introduced text is not a URL, stopping...")
        st.stop()


def predict():
    input_text = st.text_area("Enter text") #introducimos el texto
    if input_text != "":
        payload = {"text": input_text} #si el texto es distinto de vacío lo metemos en un diccionario
        try:
            response = requests.post(session_state.url, json=payload) #realizamoms la petición post a la URL
            prediction = response.json() #extraemos la información de respuesta en un json
            prediction["text"] = input_text #metemos el texto en el json
            if session_state.predictions is None: #si la predicción exixte la vacíamos en un dataframe de salida
                session_state.predictions = pd.DataFrame.from_dict(
                    {k: [v] for k, v in prediction.items()}
                )
            else:
                session_state.predictions = session_state.predictions.append(
                    {k: v for k, v in prediction.items()}, ignore_index=True
                )
            st.table(session_state.predictions)
            st.balloons()
        except Exception as ex:
            st.error(repr(ex))
        st.stop()


st.title("Sentiment Analysis Predictions")

st.markdown(
    "Welcome! With this app you can predict the sentiment of a given text using Deep Learning :smile:"
)

#cacheamos la URL en caso de que dispongamos de ella
if session_state.url is None:
    st.write("Fist, paste below the predictor server URL: ")
else:
    st.write("Using cached server URL, change if desired:")

server_url = st.text_input(
    "Server URL", value=(session_state.url if session_state.url is not None else "")
)
session_state.url = server_url

if session_state.url is not None and server_url != "": # en caso de disponer de ella validamos que el texto se trata de un URL
    validate_url(server_url)
    predict()


Overwriting front.py


Levantamos el nuevo front:

In [ ]:
import nest_asyncio
from pyngrok import ngrok, conf

conf.get_default().auth_token = ""

ngrok_tunnel = ngrok.connect(8501)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

Public URL: http://73cf-104-196-115-8.ngrok.io


In [ ]:
! streamlit run front.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://104.196.115.8:8501

2023-03-14 22:01:38.043 `st.cache` is deprecated. Please use one of Streamlit's new caching commands,
`st.cache_data` or `st.cache_resource`.

More information [in our docs](https://docs.streamlit.io/library/advanced-features/caching).
2023-03-14 22:02:40.235 `st.cache` is deprecated. Please use one of Streamlit's new caching commands,
`st.cache_data` or `st.cache_resource`.

More information [in our docs](https://docs.streamlit.io/library/advanced-features/caching).
2023-03-14 22:02:50.180 `st.cache` is deprecated. Please use one of Streamlit's new caching commands,
`st.cache_data` or `st.cache_resource`.

More information [in our docs](https://docs.streamlit.io/library/advanced-features/caching).
  Stopping...
^C


Podemos pararlo por el momento.

# Despliegue en GCP de la aplicación

Ahora que ya tenemos nuestro servicio de inferencia online funcionando, estamos listos para desplegar a escala para que soporte miles de peticiones de manera concurrente, para ello utilizaremos el servicio de GCP Cloud Run.

Cloud Run es un servicio Serverless (o sin servidor). Serveless nos facilita la puesta en producción de aplicaciones pues, en este caso GCP, se encarga de gestionar la infraestructura y recursos desplegados para nuestra aplicación en base a la carga que tenga esta misma a lo largo del tiempo. 

Esta práctica nos permita escalar de manera casi infinita, desde dar servicio desde tan solo a decenas de usuarios como a millones de manera concurrente sin tener que realizar ningún ajuste.

Las aplicaciones Serverless son una alternativa a los microservicios y los monolitos.

## Creando una aplicacion Serverless

Para crear nuestra aplicación serverless tan solo nos tenemos que preocupar de que nuestro código funciona y empaquetar todo en una imagen Docker que, finalmente, será lo que despleguemos Cloud Run.

Las aplicaciones serverless nos permiten olvidarnos de la gestión de la infrastructura. Basta con implementar el microservicio y Google gestionará toda la infra en torno al mismo.

Para crear esta imagen, dado que estamos en un entorno de Google Colab no podemos usar Docker, haremos uso del servicio Cloud Build en GCP, que se encargará de generarnos la imagen con nuestro Dockerfile de la aplicación y finalmente la guardará en el Google Container Registry, donde se almacenan las imágenes Docker. Cloud Build genera un ciclo de integración continua al que podemos enviar trabajos que generan imágenes de Docker.

In [ ]:
%cd /content/TwitterOnline

/content/TwitterOnline


In [ ]:
!ls

app  app.py  Dockerfile  model.h5  README.md  requirements.txt	tokenizer.pkl


Subimos el trabajo. Le indicamos la ruta. Coge por defecto el dockerfile y llamas a la imagen con el parámetro que indicamos en el tag.

In [ ]:
! gcloud builds submit --tag gcr.io/$PROJECT_ID/sentiment-analysis-server

Creating temporary tarball archive of 44 file(s) totalling 355.5 MiB before compression.
Uploading tarball of [.] to [gs://puestaproduccionkcmarzo23_cloudbuild/source/1678395336.207541-2eac6fbd42d7413b8d426b3ab03b15d9.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/puestaproduccionkcmarzo23/locations/global/builds/b799d99e-8c4d-4e1e-99ce-ddd828a39401].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/b799d99e-8c4d-4e1e-99ce-ddd828a39401?project=1023061570804 ].
 REMOTE BUILD OUTPUT
starting build "b799d99e-8c4d-4e1e-99ce-ddd828a39401"

FETCHSOURCE
Fetching storage object: gs://puestaproduccionkcmarzo23_cloudbuild/source/1678395336.207541-2eac6fbd42d7413b8d426b3ab03b15d9.tgz#1678395357396826
Copying gs://puestaproduccionkcmarzo23_cloudbuild/source/1678395336.207541-2eac6fbd42d7413b8d426b3ab03b15d9.tgz#1678395357396826...
- [1 files][ 40.8 MiB/ 40.8 MiB]                                                
Operation completed over 1 objects/40.8 MiB.
BUI

Finalmente, procederemos a desplegar la imagen docker en el servicio de GCP Cloud Run.

Para ello buscamos Cloud Run en las herramientas de Cloud, habilitamos la API. Vamos a crear servicio. Elegimos la imagen docker que hemos construido. Le damos un nombre e indicamos donde desplegarlo.

En memoria ponemos 8GB con 4 núcleos.

Marcamos primera generación en entoros de ejecución.

En variables de entorno agregamos: DEFAULT_MODEL_PATH : gs://content/TwitterOnline/data/model

Una vez levantado nos va a generar una URL.


## Probando nuestra aplicación desplegada

Una vez desplegado nuestra aplicación serverless de inferencia online... ¡Podemos usarla desde cualquier lugar del planeta! Tanto si es un usuario como si son millones. Para hacer un ejemplo de petición a nuestro modelo desplegado, podemos hacer lo siguiente:

In [ ]:
! curl -X POST "https://sentiment-analysis-server-xj4lbza6gq-ew.a.run.app/api/model/predict" -H  "accept: application/json" -H  "Content-Type: application/json" -d "{\"text\":\"i hate\"}"

Service Unavailable